<a href="https://colab.research.google.com/github/athipan1/Database_painaidee/blob/main/Database_Painaidee_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🏛️ Database Painaidee - Thai Attractions API

## ระบบจัดการข้อมูลสถานที่ท่องเที่ยวไทย / Thai Attractions Database Management System

This notebook allows you to run the Database Painaidee Flask API on Google Colab with ngrok tunneling for external access.

**What You Get:**
- 🏛️ **Real Thai tourism data** loaded from Tourism Authority of Thailand (TAT)
- 🌐 **Public API access** via ngrok tunnel
- 📊 **Interactive dashboard** for data visualization
- 🗃️ **Full database** with PostgreSQL and Redis
- ⚡ **Complete API** with all endpoints functional

**Features:**
- 🏛️ Thai attractions database management
- 🤖 AI-powered features for content analysis
- 📊 Behavior analytics and user tracking
- 🗺️ Auto-geocoding for location data
- ⚡ Real-time dashboard

**Note:** This is a temporary deployment that will be accessible until this Colab session ends.

## 🚀 Quick Start Guide

**To get your Thai tourism API running in 5 minutes:**

1. **Run All Cells**: Click `Runtime → Run all` to execute all cells automatically
2. **Wait for Setup**: The system will install dependencies and set up the database (~3-4 minutes)
3. **Get Your URL**: Look for the ngrok public URL in the output (e.g., `https://abc123.ngrok.io`)
4. **Data Loads Automatically**: Real Thai attraction data from TAT will be loaded into the database
5. **Start Using**: Your API and dashboard will be immediately accessible!

**What you'll get:**
- 🌐 Public API URL for immediate access
- 🏛️ Database with real Thai tourism attractions
- 📊 Web dashboard at `[your-url]/api/dashboard/`
- 🔗 API endpoints for attractions, statistics, and more

**Ready? Click `Runtime → Run all` to start!**

## 🛠️ Setup and Installation

In [ ]:
# Clone the repository
!git clone https://github.com/athipan1/Database_painaidee.git
%cd Database_painaidee

In [ ]:
# Install required Python packages
!pip install flask flask-sqlalchemy psycopg2-binary celery redis python-dotenv requests gunicorn
!pip install matplotlib seaborn gradio

# Install ngrok for tunneling
!pip install pyngrok

# Install PostgreSQL for local database
!apt update
!apt install -y postgresql postgresql-contrib redis-server

## 🗃️ Database Setup

In [ ]:
# Setup PostgreSQL
import os
import subprocess
import time

# Start PostgreSQL service
!service postgresql start

# Setup database user and database
!sudo -u postgres createuser --superuser root
!sudo -u postgres createdb painaidee_db

# Start Redis service
!service redis-server start

print("✅ Database services started")

In [ ]:
# Create environment configuration
env_content = """
FLASK_ENV=development
SECRET_KEY=colab-demo-secret-key-123
DATABASE_URL=postgresql://root@localhost:5432/painaidee_db
POSTGRES_DB=painaidee_db
POSTGRES_USER=root
POSTGRES_PASSWORD=
REDIS_URL=redis://localhost:6379/0
CELERY_BROKER_URL=redis://localhost:6379/0
DEBUG=True
PAGINATION_ENABLED=true
PAGINATION_PAGE_SIZE=20
API_TIMEOUT=30
"""

with open('.env', 'w') as f:
    f.write(env_content.strip())

print("✅ Environment configuration created")

## 🚀 Initialize Application

In [ ]:
# Initialize the database
!python init_db.py
print("✅ Database initialized")

## 🌐 Setup ngrok Tunnel

ngrok will create a public URL to access your Flask app running on Colab.

In [ ]:
from pyngrok import ngrok
import threading
import time

# Set up ngrok tunnel
port = 5000
public_url = ngrok.connect(port)
print(f"\n🌐 ngrok tunnel created!")
print(f"📱 Access your API at: {public_url}")
print(f"📊 Dashboard available at: {public_url}/api/dashboard/")
print(f"ℹ️  API information at: {public_url}/")
print(f"\n⚠️  Note: This URL will be active until this Colab session ends.")

## ▶️ Run the Flask Application

In [ ]:
# Start Celery worker in background
import subprocess
import threading
import os

def start_celery():
    os.system('celery -A tasks.celery worker --loglevel=info --detach')

# Start Celery worker
celery_thread = threading.Thread(target=start_celery)
celery_thread.daemon = True
celery_thread.start()

print("✅ Celery worker started")
time.sleep(3)

In [ ]:
# Run the Flask application
from app import create_app
import threading

# Create Flask app
flask_app = create_app()

def run_flask():
    flask_app.run(
        host='0.0.0.0',
        port=port,
        debug=False,  # Disable debug in threading
        use_reloader=False,  # Disable reloader in Colab
        threaded=True
    )

# Start Flask in a separate thread
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

# Wait a moment for the server to start
time.sleep(5)

print(f"\n🚀 Flask application is now running!")
print(f"\n📱 **Access your API:**")
print(f"   Main API: {public_url}")
print(f"   Dashboard: {public_url}/api/dashboard/")
print(f"   Health Check: {public_url}/api/health")
print(f"   Attractions: {public_url}/api/attractions")
print(f"\n🔗 **API Endpoints to try:**")
print(f"   GET  {public_url}/api/attractions")
print(f"   POST {public_url}/api/attractions/sync")
print(f"   GET  {public_url}/api/dashboard/stats")
print(f"\n⏰ Keep this cell running to maintain the server...")

## 🧪 Test the API

In [ ]:
import requests
import json

# Test API endpoints
base_url = str(public_url)

print("🧪 Testing API endpoints...\n")

# Test health endpoint
try:
    response = requests.get(f"{base_url}/api/health")
    print(f"✅ Health Check: {response.status_code}")
    print(f"   Response: {response.json()}\n")
except Exception as e:
    print(f"❌ Health Check failed: {e}\n")

# Test main API info
try:
    response = requests.get(f"{base_url}/")
    print(f"✅ API Info: {response.status_code}")
    if response.status_code == 200:
        data = response.json()
        print(f"   Message: {data.get('message', 'N/A')}")
        print(f"   Version: {data.get('version', 'N/A')}\n")
except Exception as e:
    print(f"❌ API Info failed: {e}\n")

# Test attractions endpoint
try:
    response = requests.get(f"{base_url}/api/attractions")
    print(f"✅ Attractions: {response.status_code}")
    if response.status_code == 200:
        data = response.json()
        print(f"   Total attractions: {len(data.get('attractions', []))}")
except Exception as e:
    print(f"❌ Attractions failed: {e}")

print(f"\n🌐 Your API is accessible at: {public_url}")
print(f"📊 Dashboard: {public_url}/api/dashboard/")

## 📊 Load Real Thai Tourism Data

In [ ]:
# Load real Thai tourism data from Tourism Authority of Thailand (TAT)
print("🏛️ Loading Thai tourism data from TAT Open Data...")
print("This will populate the database with real Thai attractions data.\n")

try:
    # First, try to load real TAT CSV data (preferred)
    tat_payload = {
        "csv_url": "https://opendata.tourismthailand.org/data/attractions.csv",
        "enable_geocoding": False  # Disable geocoding for faster loading in Colab
    }
    
    print("📋 Loading TAT Open Data CSV (Real Thai Attractions)...")
    response = requests.post(f"{base_url}/api/attractions/sync/tat", json=tat_payload)
    
    if response.status_code == 200:
        result = response.json()
        print(f"✅ TAT Data Loaded Successfully!")
        print(f"   📊 Saved: {result.get('saved', 0)} attractions")
        print(f"   🔄 Updated: {result.get('updated', 0)} attractions")
        print(f"   ⏭️ Skipped: {result.get('skipped', 0)} attractions")
        print(f"   📈 Total Processed: {result.get('total_processed', 0)} attractions")
        
        if result.get('saved', 0) > 0 or result.get('updated', 0) > 0:
            print(f"\n🎉 Database now contains real Thai tourism data!")
        else:
            print(f"\n⚠️ No new data added (may already exist in database)")
    else:
        print(f"⚠️ TAT CSV loading failed ({response.status_code}), trying fallback...")
        raise Exception(f"TAT API returned status {response.status_code}")
        
except Exception as e:
    print(f"\n🔄 TAT data unavailable, using test data for demonstration...")
    print(f"Error details: {e}")
    
    # Fallback to external API sync (test data)
    try:
        response = requests.post(f"{base_url}/api/attractions/sync")
        if response.status_code == 200:
            result = response.json()
            print(f"\n✅ Fallback Test Data Loaded:")
            print(f"   💾 Saved: {result.get('saved', 0)} items")
            print(f"   ⏭️ Skipped: {result.get('skipped', 0)} items")
            print(f"   📈 Total: {result.get('total_processed', 0)} items")
        else:
            print(f"❌ Fallback sync also failed: {response.status_code}")
    except Exception as fallback_error:
        print(f"❌ Both data sources failed: {fallback_error}")

# Verify data was loaded
try:
    response = requests.get(f"{base_url}/api/attractions")
    if response.status_code == 200:
        data = response.json()
        attraction_count = len(data.get('attractions', []))
        print(f"\n📊 Database Status: {attraction_count} attractions loaded")
        
        if attraction_count > 0:
            # Show sample attraction
            sample = data['attractions'][0]
            print(f"\n🏛️ Sample Attraction:")
            print(f"   Name: {sample.get('title', 'N/A')[:60]}...")
            print(f"   Province: {sample.get('province', 'N/A')}")
            print(f"   Category: {sample.get('category', 'N/A')}")
        else:
            print(f"\n⚠️ No attractions found in database")
    else:
        print(f"\n❌ Could not verify database contents")
except Exception as e:
    print(f"\n❌ Error checking database: {e}")

print(f"\n🌐 Your API with data is accessible at: {public_url}")
print(f"📊 Dashboard: {public_url}/api/dashboard/")
print(f"🔍 Browse attractions: {public_url}/api/attractions")

## 🚀 Quick Examples & Usage

Now that your database is populated, here are some quick examples of what you can do:

In [ ]:
# Example API calls with your populated database
import requests
import json

print("🎯 API Examples with Real Data:\n")

# 1. Get attractions count
try:
    response = requests.get(f"{base_url}/api/attractions")
    if response.status_code == 200:
        data = response.json()
        print(f"📊 Total attractions: {len(data.get('attractions', []))}")
        
        # Show attractions by province (if data available)
        if data.get('attractions'):
            provinces = {}
            for attraction in data['attractions'][:20]:  # Sample first 20
                province = attraction.get('province', 'Unknown')
                provinces[province] = provinces.get(province, 0) + 1
            
            print(f"\n🗺️ Attractions by Province (sample):")
            for province, count in sorted(provinces.items())[:10]:
                print(f"   {province}: {count} attractions")
                
except Exception as e:
    print(f"❌ Error: {e}")

# 2. Dashboard statistics
try:
    response = requests.get(f"{base_url}/api/dashboard/stats")
    if response.status_code == 200:
        stats = response.json()
        print(f"\n📈 Dashboard Statistics:")
        print(f"   Database attractions: {stats.get('total_attractions', 0)}")
        print(f"   Recent syncs: {stats.get('recent_syncs', 0)}")
        if stats.get('latest_sync'):
            print(f"   Latest sync: {stats['latest_sync'].get('created_at', 'Unknown')}")
except Exception as e:
    print(f"❌ Dashboard stats error: {e}")

# 3. Health check
try:
    response = requests.get(f"{base_url}/api/health")
    if response.status_code == 200:
        health = response.json()
        print(f"\n❤️ System Health: {health.get('message', 'Unknown')}")
except Exception as e:
    print(f"❌ Health check error: {e}")

print(f"\n🌐 **Your API Endpoints:**")
print(f"   🏠 Main API: {public_url}")
print(f"   📊 Dashboard: {public_url}/api/dashboard/")
print(f"   🏛️ Attractions: {public_url}/api/attractions")
print(f"   ❤️ Health: {public_url}/api/health")
print(f"   📈 Stats: {public_url}/api/dashboard/stats")

print(f"\n💡 **Try these in your browser or with curl:**")
print(f"   curl {public_url}/api/health")
print(f"   curl {public_url}/api/dashboard/stats")

## 🔄 Keep Server Running

**Important:** Keep the cell below running to maintain your server. 

The API will be accessible at the ngrok URL until you stop this cell or the Colab session ends.

In [ ]:
# Keep the server running
import time

print(f"🚀 Server is running at: {public_url}")
print(f"📊 Dashboard: {public_url}/api/dashboard/")
print(f"\n⏰ Server will keep running...")
print(f"\n🛑 To stop the server, interrupt this cell (Runtime → Interrupt execution)")

try:
    while True:
        time.sleep(30)
        print(f"⚡ Server alive at {time.strftime('%H:%M:%S')} - {public_url}")
except KeyboardInterrupt:
    print("\n🛑 Server stopped by user")
    ngrok.disconnect(public_url)
    print("🔌 ngrok tunnel disconnected")

## 📖 Usage Guide

### 🎯 What You Just Built
You now have a fully functional Thai tourism API running on Google Colab with:
- **Real Tourism Data**: Loaded from Tourism Authority of Thailand (TAT)
- **Public Access**: Available via ngrok tunnel to anyone with the URL
- **Complete Database**: PostgreSQL with attractions, geocoding, and analytics
- **Background Processing**: Celery + Redis for async operations
- **Web Dashboard**: Real-time monitoring and statistics

### 🔗 API Endpoints Available

**Core Endpoints:**
- `GET /` - API information and features overview
- `GET /api/health` - Health check and system status
- `GET /api/attractions` - Get all Thai attractions in database
- `POST /api/attractions/sync` - Sync additional test data
- `POST /api/attractions/sync/tat` - Sync more TAT CSV data

**Dashboard & Analytics:**
- `GET /api/dashboard/` - Interactive web dashboard
- `GET /api/dashboard/stats` - System statistics
- `GET /api/dashboard/health` - Detailed health information

**AI Features (if available):**
- `POST /api/ai/keywords/extract` - Extract keywords from attraction data
- `GET /api/ai/recommendations/{user_id}` - Get AI recommendations
- `POST /api/behavior/track` - Track user interactions

### 💻 Sample Usage Examples

**Python/Requests:**
```python
import requests

# Your ngrok URL from above
api_url = "YOUR_NGROK_URL_HERE"

# Get all attractions
response = requests.get(f"{api_url}/api/attractions")
attractions = response.json()
print(f"Found {len(attractions['data'])} Thai attractions")

# Get system stats
stats = requests.get(f"{api_url}/api/dashboard/stats").json()
print(f"Database has {stats['total_attractions']} attractions")
```

**cURL Commands:**
```bash
# Health check
curl YOUR_NGROK_URL/api/health

# Get attractions
curl YOUR_NGROK_URL/api/attractions | jq .

# Get dashboard stats
curl YOUR_NGROK_URL/api/dashboard/stats | jq .
```

**JavaScript/Fetch:**
```javascript
// Get attractions
fetch('YOUR_NGROK_URL/api/attractions')
  .then(response => response.json())
  .then(data => console.log('Attractions:', data.data.length));
```

### 🌐 Web Interface
- **Dashboard**: Visit `YOUR_NGROK_URL/api/dashboard/` for a web interface
- **API Info**: Visit `YOUR_NGROK_URL/` for API documentation
- **Direct Data**: Visit `YOUR_NGROK_URL/api/attractions` to see JSON data

### 🚀 Next Steps
1. **Explore the Dashboard**: Visit the dashboard URL for visual analytics
2. **Test API Endpoints**: Use the URLs above with curl, Postman, or code
3. **Build Applications**: Use this API as backend for mobile/web apps
4. **Analyze Data**: Query the attractions for tourism insights

### ⚠️ Important Notes
- 🕒 **Temporary**: This deployment lasts only during your Colab session
- 🌐 **Public**: The ngrok URL is accessible by anyone with the link
- 🔄 **Persistent**: For permanent deployment, use Hugging Face Spaces or deploy locally
- 💾 **Data**: Real Thai tourism data is loaded and ready to use

### 🔗 For Production Use
- **Hugging Face Spaces**: Use the Deploy button in the main repository for permanent hosting
- **Local Docker**: Clone the repo and run with `docker-compose up -d`
- **Full Repository**: Visit https://github.com/athipan1/Database_painaidee for complete documentation